In [ ]:
%%capture
!pip install torch
!pip install -r requirements.txt
!pip install selenium

In [1]:
from ddpg_torch import AgentDDPG
import gym 
import numpy as np
from utils import plotLearning
import os

import glob
from PIL import Image

from utils import show_video, convert_gif
from gym.wrappers.monitoring.video_recorder import VideoRecorder

    
from base64 import b64encode

In [2]:
name_env = 'LunarLanderContinuous-v2'
# name_env ="BipedalWalker-v3"
env = gym.make(name_env)

In [3]:
print([*env.observation_space.shape],*env.action_space.shape)

input_dims = [*env.observation_space.shape]
n_actions = env.action_space.shape[0]

print(input_dims,n_actions)

[8] 2
[8] 2


In [ ]:
def make_gif(frame_folder,name):
    frames = [Image.open(image) for image in glob.glob(f"{frame_folder}/*.png")]
    frame_one = frames[0]
    frame_one.save(os.path.join(frame_folder,f"{name}.gif"), format="GIF", append_images=frames,
               save_all=True, duration=100, loop=0)    
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

In [ ]:
agent = AgentDDPG(
    alpha=0.000925,
    beta=0.0000915,
    input_dims=input_dims,
    tau=0.001,env=env,
    rollout_len = 500,
    total_rollouts = 1000,
    batch_size=64,
    layer1_size=400,
    layer2_size=300,
    n_actions=n_actions,
    agent_dir=os.path.join(os.getcwd(),name_env))
    

In [ ]:
agent.train(True,plot_save=os.path.join(os.getcwd(),'plots',name_env))

In [ ]:
make_gif(os.path.join(os.getcwd(),'plots',name_env),name_env)

In [ ]:
from pyvirtualdisplay import Display
from selenium import webdriver

display = Display(visible=0, size=(800, 600))
display.start()

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')  # May be required in some environments
chrome_options.add_argument('--disable-dev-shm-usage')  # May be required in some environments
chrome_options.add_argument('--headless')

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# videos_dir = os.path.join(f"{name_env}",f"{name_env}_before_training.mp4")
# # if not os.path.exists(videos_dir):
# #     # Create a new directory because it does not exist
# #     os.makedirs(videos_dir)
# #     print("The new directory is created!")
    
# before_training = videos_dir

# video = VideoRecorder(env, before_training)
# # returns an initial observation
# env.reset()
# for i in range(200):
#   env.render()
#   video.capture_frame()
#   # env.action_space.sample() produces either 0 (left) or 1 (right).
#   new_state,reward,done,info,_ =  env.step(env.action_space.sample())
#   # Not printing this time
#   #print("step", i, observation, reward, done, info)

# video.close()
# env.close()

before_training = "before_training.mp4"

video = VideoRecorder(env, before_training)
# returns an initial observation
env.reset()
for i in range(200):
  env.render()
  video.capture_frame()
  # env.action_space.sample() produces either 0 (left) or 1 (right).
  act = env.action_space.sample() 
  observation, reward, done, info,_ = env.step(act)
  # Not printing this time
  #print("step", i, observation, reward, done, info)

video.close()
env.close()

In [ ]:
from IPython.display import HTML
html = render_mp4(before_training)
HTML(html)

In [ ]:
if not os.path.exists(plot_save):
    # Create a new directory because it does not exist
    os.makedirs(plot_save)
    print("The new directory is created!")
for i in range(1000):
    done = False
    score = 0
    obs = env.reset()[0]
    # print(obs)
    while not done:
        act = agent.choose_action(obs) 
        new_state,reward,done,info,_ = env.step(act)
        agent.remember(state=obs,action=act,reward=reward,new_state=new_state,done=done)
        
        agent.learn()
        score += reward
        obs = new_state
    score_history.append(score)
    print(f'Episode {i} score {round(score, 2)} 100 game average {round(np.mean(score_history[-100:]), 2)}')
    if i % 25 == 0:
        agent.save_models()   
    filename = f'{i}.png'
    plotLearning(score_history,os.path.join(plot_save,filename),window=100)
make_gif(plot_save,name_env)

In [ ]:
from gym.wrappers.monitoring.video_recorder import VideoRecorder
before_training = "before_training.mp4"

video = VideoRecorder(env, before_training)
# returns an initial observation
env.reset()
for i in range(200):
  env.render()
  video.capture_frame()
  # env.action_space.sample() produces either 0 (left) or 1 (right).
  observation, reward, done, info = env.step(env.action_space.sample())
  # Not printing this time
  #print("step", i, observation, reward, done, info)

video.close()
env.close()

In [ ]:
for env_name in [
    'LunarLanderContinuous-v2',
    "BipedalWalker-v3"]:
    env = gym.make(env_name)
    input_dims = [*env.observation_space.shape]
    n_actions = env.action_space.shape[0]   
    
    agent = AgentDDPG(
    alpha=0.000025,
    beta=0.00025,
    input_dims=input_dims,
    tau=0.001,env=env,
    batch_size=64,
    layer1_size=400,
    layer2_size=300,
    n_actions=n_actions,
    agent_dir=os.path.join(os.getcwd(),env_name))
    
    
    
    np.random.seed(42)
    score_history = []
    plot_save = os.path.join(os.getcwd(),'plots',env_name)
    
    
    if not os.path.exists(plot_save):
        # Create a new directory because it does not exist
        os.makedirs(plot_save)
        print("The new directory is created!")
    for i in range(1000):
        done = False
        score = 0
        obs = env.reset()[0]
        # print(obs)
        while not done:
            act = agent.choose_action(obs) 
            new_state,reward,done,info,_ = env.step(act)
            agent.remember(state=obs,action=act,reward=reward,new_state=new_state,done=done)
            agent.learn()
            score += reward
            obs = new_state
        score_history.append(score)
        print(f'Episode {i} score {round(score, 2)} 100 game average {round(np.mean(score_history[-100:]), 2)}')
        if i % 25 == 0:
            agent.save_models()   
        filename = f'{i}.png'
        plotLearning(score_history,os.path.join(plot_save,filename),window=100)
    make_gif(plot_save,env_name)